#  주택청약 FAQ 시스템 챗봇 구현 

- 문서 전처리 + RAG + Gradio ChatInterface

## 1. 환경 설정

In [1]:
# 환경 변수
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 기본 라이브러리
import os
from glob import glob

from pprint import pprint
import json

In [9]:
# LLM 설정
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.8, 
)

## 2. 문서 전처리

* 데이터 정제 : 원본 문서에서 HTML 태그, 특수문자, 중복 문장 등을 제거하고 텍스트를 표준화하여 검색 품질을 높임
* 문서 청킹(Chunking) : 문서를 문맥이 유지되도록 문장 또는 단락 단위로 분할하여 검색 정확도와 응답 품질을 향상시킴
* 임베딩(Embedding) : 텍스트를 의미 기반 고차원 벡터로 변환하여 유사도 기반 검색이 가능하도록 함
* 벡터 데이터베이스 색인화 : 임베딩된 벡터를 벡터 데이터베이스에 색인화하여 대규모 문서에서도 빠른 검색을 가능하게 함

### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공한 문서로, 50개의 문답이 포함된 텍스트 파일임

In [5]:
# 여기에 코드를 작성하세요.
from langchain_community.document_loaders import TextLoader

loader = TextLoader(
    file_path="data/housing_faq.txt",
    encoding="utf-8"
)
docs = loader.load()

In [6]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [8]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': 'data/housing_faq.txt'}

### 2) 문서 전처리

(1) 정규표현식을 활용하여 문서에서 질문-답변 쌍을 구조적으로 분리함

In [7]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [8]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


(2) LLM을 활용하여 텍스트에서 키워드와 핵심 개념을 추출하고, 이를 메타데이터나 본문에 추가하여 검색 성능을 향상시킴

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="질문 텍스트에서 추출한 가장 중요한 키워드")
    summary: str = Field(description="질문과 응답 텍스트의 핵심 내용 요약")

# 프롬프트 템플릿 정의
template = """주어진 질문 텍스트에서 핵심 단어를 추출하고, 응답 텍스트의 핵심 내용을 요약합니다.
질문 및 응답 텍스트의 맥락을 고려하여 핵심 용어나 전문 용어, 주요 아이디어나 원리 등을 추출합니다.

질문 텍스트:
{question}

응답 텍스트:
{answer}

JSON 형식으로 다음 정보를 반환하시오:
- keyword: 단어 1개 만
- summary: 1-2 문장 만
"""

# LCEL 체인 구성 (Sturctured Output 사용)
prompt = ChatPromptTemplate.from_template(template)
llm_with_structure = llm.with_structured_output(KeywordOutput)
keyowrd_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke({
    "question": qa_pairs[36]['question'],
    "answer": qa_pairs[36]['answer']
})
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 입주자저축
요약: 제2순위 청약신청 시 입주자저축 통장이 필요하며, 분양주택 또는 분양전환공공임대주택 입주자로 선정된 경우 해당 통장은 재사용할 수 없습니다.


(3) 요약문을 시맨틱 검색에 활용하기 위해, 요약을 `page_content`에 담고 기타 정보를 `metadata`로 구성한 문서 객체를 생성함.

In [22]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:
        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke({
            "question": pair['question'],
            "answer": pair['answer']
        })

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[5].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[5].metadata)

포맷팅된 문서 개수: 50
대규모택지개발지구에서 주택 공급 시 특별공급 물량도 공급 비율에 따라 배정되며, 거주지별 우선공급 비율이 적용된다. 다자녀 특별공급은 별도의 운용지침에 따라 시·도 단위 우선공급 후 수도권 거주자에게 공급된다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '공급규칙 제34조가 적용되는 지역에 주택을 공급하는 경우 특별공급 물량 또한 그 공급 비율에 따라 배정하여야 합니다. '
           '(예) 서울특별시·인천광역시 대규모 택지개발지구에서 공급되는 주택: 특별시·광역시 거주자(거주기간 충족 필요)에게 '
           '공급물량의 50% 우선공급 → 이후 잔여물량을 전체 수도권 거주자를 대상으로 공급 (예) 경기도 과천시 대규모 '
           '택지개발지구에서 공급되는 주택: 과천시 거주자(거주기간 충족 필요)에게 공급물량의 30% 우선공급 → 이후 과천시 공급 '
           '잔여물량 + 20% 경기도 거주자(거주기간 충족 필요)에게 공급 → 이후 잔여물량을 전체 수도권 거주자를 대상으로 공급 '
           '다만, 다자녀 특별공급의 경우 「다자녀가구 및 노부모부양 주택 특별공급 운용지침」 제5조 단서에 따라 수도권에서 '
           '입주자를 모집하는 때에는 해당 주택건설지역 시·군·구가 속한 시·도에 50퍼센트를 우선공급하고 나머지 주택(우선공급에서 '
           '미분양된 주택을 포함한다)은 수도권 거주자(우선공급에서 입주자로 선정되지 아니한 자를 포함한다)에게 공급할수 있습니다. '
           '(예) 경기도 과천시: 경기도 거주

In [21]:
# 문서 저장
output_file = "data/housing_faq_formatted.json"
with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump(
        [doc.model_dump() for doc in formatted_docs], 
        f, 
        indent=2, 
        ensure_ascii=False
    )  # 한글이 유니코드로 변환되지 않도록 설정

print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

포맷팅된 문서를 data/housing_faq_formatted.json에 저장했습니다.


In [ ]:
# 문서 로드
from langchain_core.documents import Document

output_file = "data/housing_faq_formatted.json"
with open(output_file, 'r', encoding='utf-8-sig') as f:
    formatted_docs = [Document(**doc) for doc in json.load(f)]
    
# 문서 확인
print(formatted_docs[0].page_content)

## 3. 벡터 저장 

### 1\) Store 생성

In [ ]:
from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": "./milvus.db"},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)



# 문서 검색

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

In [ ]:
# 여기에 코드를 작성하세요.
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold":0.05},
)

# 벡터 저장소 로드
vector_store_with_summary = None

In [ ]:
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

MMR 검색기 정의

- 요약 문서 벡터 스토어를 사용
- 10개의 문서를 가져와서, 다양성 기반으로 3개를 선택 (다양성은 중간 수준 적용)

In [ ]:
# 여기에 코드를 작성하세요.
mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

### **[심화] 메타데이터 기반 필터링**

- Chroma 문서: https://docs.trychroma.com/docs/querying-collections/metadata-filtering

In [ ]:
# 단일 필드 정확히 일치
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": "주택건설지역"}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $eq 연산자 사용 - 정확히 일치

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$eq": "주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $ne (Not Equal) 연산자 사용 - 정확히 일치하지 않는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$ne": "주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $in 연산자로 여러 값 중 일치하는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$in": ["주택건설지역", "청약예금"]}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 숫자 범위 검색 ($gt, $gte, $lt, $lte) - question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"question_id": {"$gte": 10}}},
)

query = "무주택자 기준은 무엇인가요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $and로 여러 조건 조합 - keyword가 "주택건설지역"이고 question_id가 10 미만인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$and": [
        {"keyword": "주택건설지역"}, 
        {"question_id": {"$lt": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $or로 여러 조건 중 하나 일치하는 문서 검색 - keyword가 "주택건설지역"이거나 question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$or": [
        {"keyword": "주택건설지역"}, 
        {"question_id": {"$gte": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 정규식 패턴 매칭 - page_content 본문에 "주택건설지역"이 포함된 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={'where_document': {'$contains': '주택건설지역'}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional

class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    question_id: Optional[int] = Field(description="질문 ID의 최소값")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
다음 예시를 참조한다:

1. 키워드 검색 예시:
- 입력: "주택건설 관련 문서 찾아줘"
출력:
keyword: "주택건설"
keyword_expression: "$eq"

2. 질문 ID 검색 예시:
- 입력: "질문 ID 10번 이상인 문서"
출력:
question_id: 10
question_id_expression: "$gte"

3. 복합 검색 예시:
- 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
출력:
keyword: "주택건설"
keyword_expression: "$eq"
question_id: 5 
question_id_expression: "$gte"

검색 표현식은 다음과 같이 사용한다:
- 동등 비교: $eq
- 크거나 같음: $gte
- 작거나 같음: $lte
- 범위 검색: $gt, $lt

요청에 해당 정보가 없는 경우 해당 필드는 null로 반환한다.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{query}")
])

# 구조화된 출력을 위한 체인 생성
model_with_structure = llm.with_structured_output(MetadataFilter)
metadata_chain = prompt | model_with_structure

# 사용 예시
query = "주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"
filter_params = metadata_chain.invoke({"query": query})

print(f"키워드: {filter_params.keyword}")
print(f"키워드 표현식: {filter_params.keyword_expression}")
print(f"질문 ID: {filter_params.question_id}")
print(f"질문 ID 표현식: {filter_params.question_id_expression}")

In [ ]:
# 메타데이터 필터 생성
filter_dict = {}

if filter_params.keyword and filter_params.question_id:
    # 두 조건 모두 있는 경우 AND 연산자 사용
    filter_dict = {
        "$and": [
            {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
            {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
        ]
    }
elif filter_params.keyword:
    # 키워드 조건만 있는 경우
    filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
elif filter_params.question_id:
    # 질문 ID 조건만 있는 경우
    filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

# retriever에 필터 적용 
retriever = vector_store.as_retriever(
    search_kwargs={"filter": filter_dict} if filter_dict else {}
)

# 검색 실행
query = "주택건설지역 관련 문서를 질문 ID 10번 이하인 문서중에서 검색해주세요"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from pydantic import BaseModel, Field
from typing import Optional, Dict


class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    question_id: Optional[int] = Field(description="질문 ID의 최소값")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


def create_metadata_filter(query: str, llm) -> Dict:
    """
    사용자 쿼리를 분석하여 Chroma DB 검색을 위한 메타데이터 필터를 생성합니다.

    Args:
        query: 사용자 검색 쿼리.
        llm: 사용할 언어 모델.

    Returns:
        Chroma DB 검색에 사용할 수 있는 필터 딕셔너리.
    """
    system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
    다음 예시를 참조한다:

    1. 키워드 검색 예시:
    - 입력: "주택건설 관련 문서 찾아줘"
    출력:
    keyword: "주택건설"
    keyword_expression: "$eq"

    2. 질문 ID 검색 예시:
    - 입력: "질문 ID 10번 이상인 문서"
    출력:
    question_id: 10
    question_id_expression: "$lte"

    3. 복합 검색 예시:
    - 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
    출력:
    keyword: "주택건설"
    keyword_expression: "$eq"
    question_id: 5
    question_id_expression: "$gte"

    검색 표현식은 다음과 같이 사용한다:
    - 동등 비교: $eq
    - 크거나 같음: $gte
    - 작거나 같음: $lte
    - 범위 검색: $gt, $lt

    요청에 해당 정보가 없는 경우 해당 필드는 null로 반환한다.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])

    model_with_structure = llm.with_structured_output(MetadataFilter)
    metadata_chain = prompt | model_with_structure

    filter_params = metadata_chain.invoke({"query": query})

    filter_dict = {}

    if filter_params.keyword and filter_params.question_id:
        filter_dict = {
            "$and": [
                {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
                {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
            ]
        }
    elif filter_params.keyword:
        filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
    elif filter_params.question_id:
        filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

    return filter_dict



# 사용 예시 
query = "주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"

@chain
def metadata_filter_query(query: str):

    llm = ChatOpenAI(
        model='gpt-4.1-mini',
        temperature=0.1,
        top_p=0.9,
    )

    filter_dict = create_metadata_filter(query, llm)
    print(filter_dict)

    retriever = vector_store.as_retriever(
            search_kwargs={"filter": filter_dict} if filter_dict else {}
    )
    results = retriever.invoke(query)

    return results

results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 다른 쿼리
query = "청약통장에 대한 정보를 찾아주세요"
results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

# RAG Chain

### 1) 참조 문서 없이 직접 답변을 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt
template = '''Answer the question based only on the following context.

[Context]
{context}

[Question]
{question}

[Answer (in 한국어)]
'''

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)


# Chain 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
rag_chain.invoke(query)

### 2) 참조 문서를 답변과 함께 반환

`(1) 문서와 포맷팅된 컨텍스트를 함께 반환하는 함수`


In [ ]:
# 다음 코드를 완성하세요.

from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": None,  # 질문
        "context": None,   # 문서 포맷팅된 컨텍스트
        "source_documents": None   # 검색된 문서 리스트
    }

`(2) 컨텍스트와 질문을 입력으로 받아 답변을 생성하는 함수`

In [ ]:
# 다음 코드를 완성하세요.

from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = None

    return {
        "answer": None,  # 생성된 답변 (answer_chain 결과)
        "source_documents": None  # 소스 문서 정보 (input_data에서 가져옴)
    }

`(3) RAG 체인 구성`

In [ ]:
# 다음 코드를 완성하세요.

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    None(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': None(prompt_and_generate_answer), # 답변 생성
        'question': None(),
        "source_documents": None("source_documents")   # 소스 반환
    }
)

In [ ]:
# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

### 3) 검색 문서 관련성 평가

`(1) 검색 문서와 질문 간의 관련성을 평가`


In [ ]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

In [ ]:
# gpt-4.1 모델 사용

llm_gpt4o = ChatOpenAI(
    model='gpt-4.1',
    temperature=0.1,
    top_p=0.9, 
)

chain = prompt | llm_gpt4o | StrOutputParser()    # gpt-4.1 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

# Gradio 챗봇 인터페이스

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_sotre = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print("=" * 50)

In [ ]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class SearchResult:
    context: str
    source_documents: Optional[List]

class RAGSystem:
    def __init__(
            self, 
            llm: BaseChatModel, 
            eval_llm: BaseChatModel,
            retriever: VectorStoreRetriever
        ):
        if not llm:
            self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        else:
            self.llm = llm

        if not eval_llm:
            self.eval_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
        else:
            self.eval_llm = eval_llm

        if not retriever:
            raise ValueError("검색기(retriever)가 필요합니다.")
        else:
            self.retriever = retriever
        
    def _format_docs(self, docs: List) -> str:
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _format_source_documents(self, docs: Optional[List]) -> str:
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            if 'summary' in metadata:
                source_info.append(f"요약: {metadata['summary']}")
                
            formatted_docs.append(
                f"📚 참조 문서 {i}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def _check_relevance(self, docs: List, question: str) -> List:
        """문서의 관련성 확인"""

        relevant_docs = []

        if not docs:
            return relevant_docs
            
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

        다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

        1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
        2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?

        'Yes' 또는 'No'로만 답변하세요."""),
            ("human", """[컨텍스트]
        {context}

        [질문]
        {question}""")
        ])
        
        chain = prompt | self.eval_llm | StrOutputParser()

        for doc in docs:
            result = chain.invoke({
                "context": doc.page_content,
                "question": question
            }).lower()

            print(f"문서 {doc.metadata['question_id']} 관련성 확인 결과: {result}")
            print(f"문서 {doc.metadata['question_id']} 내용:")
            print(doc.page_content)
            print("-" * 50)

            if "yes" in result:
                relevant_docs.append(doc)
            
        return relevant_docs
    
    def search_documents(self, question: str) -> SearchResult:
        try:
            docs = retriever.invoke(question)
            print(f"검색된 문서 개수: {len(docs)}")
            relevant_docs = self._check_relevance(docs, question) 
            print(f"관련 문서 개수: {len(relevant_docs)}")
            
            return SearchResult(
                context=self._format_docs(relevant_docs) if relevant_docs else "관련 문서를 찾을 수 없습니다.",
                source_documents=relevant_docs,
            )
        except Exception as e:
            print(f"문서 검색 중 오류 발생: {e}")
            return SearchResult(
                context="문서 검색 중 오류가 발생했습니다.",
                source_documents=None,
            )
    
    def generate_answer(self, message: str, history: List) -> str:
        # 문서 검색
        search_result = self.search_documents(message)
        
        if not search_result.source_documents:
            return "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"
                    
        # 프롬프트 템플릿 설정
        prompt = ChatPromptTemplate.from_messages([
            ("system", """다음 지침을 따라 질문에 답변해주세요:
            1. 주어진 문서의 내용만을 기반으로 답변하세요.
            2. 문서에 명확한 근거가 없는 내용은 "근거 없음"이라고 답변하세요.
            3. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 답변하세요.
            4. 추측이나 일반적인 지식을 사용하지 마세요."""),
            ("human", "문서들:\n{context}\n\n질문: {question}")
        ])
        
        # RAG Chain 구성
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            # 답변 생성
            answer = chain.invoke({
                "context": search_result.context,
                "question": message
            })
            
            # 참조 문서 포맷팅 추가
            sources = self._format_source_documents(search_result.source_documents)
            return f"{answer}\n{sources}"
            
        except Exception as e:
            return f"답변 생성 중 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정

rag_system = RAGSystem(
    llm=ChatOpenAI(model="gpt-4.1", temperature=0),   
    eval_llm=ChatOpenAI(model="gpt-4.1", temperature=0),
    retriever=vector_store.as_retriever(search_kwargs={"k": 2})
)

demo = gr.ChatInterface(
    fn=rag_system.generate_answer,
    title="RAG QA 시스템",
    description="""
    질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
    모든 답변에는 참조한 문서의 출처가 표시됩니다.
    """,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="gray",
    ),
examples=[
    ["수원시의 주택건설지역은 어디에 해당하나요?"],
    ["무주택 세대에 대해서 설명해주세요."],
    ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
],
)

# 데모 실행
demo.launch()

In [ ]:
# Gradio 인터페이스 종료
demo.close()